In [1]:
#All Code Written by Gary Zeri
#Chapman University Computer Science Major, Member of the LaRue CatLab

#Allow Notebook to Import from Comp_Chem_Package
import sys
sys.path.append("..\\Comp_Chem_Package")

#Import All Required Packages Here
import ipywidgets as widgets
import numpy as np
from IPython.display import clear_output
from diatomicConstants import diatomicConstantsWidget
from diatomicConstants import diatomicConstants
from diatomicPotentials import extendedRydberg
from wavefunction import wavefunction
from basisSets import how
from rkr import rkr
from color import getColor

from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [2]:
#Declare All Global Variables Here
figure = {
        "data": [],
        "layout":
        {
           "xaxis":{"title":"Bond Distance"},
           "yaxis":{"title":"Energy"},
           "title":{"text":"Potential Energy Curves"}
        },    
    }    

rydberg = extendedRydberg()
rkr = rkr()
DCW = 0

In [3]:
##########################################################################################

def plot(x, y, name, lineColor="", dash="solid", mode="lines"):
    
    if(lineColor == ""):
        lineColor = getColor()
    
    figure["data"].append(
         {
            "type":"scatter",
            "x":x,
            "y":y,
           # "connectgaps":True,
            "mode":mode, 
            "name":name,
             "line":{"dash":dash, "color":lineColor}
        }
    )
    
##########################################################################################
    
def buildWavefunction():
    
    #reD=1.12832
    
    #Actually Hydrogen
    #CO = diatomicConstants(T=0, re=0.74144 , w=4401.21, wx=121.33, wy=0, wz=0, B=60.853, a=3.062, 
                           #y=0, D=0.0471, u=0.5, state="ground") 
    CO = diatomicConstants(T=0, re=1.128323, w=2169.81358, wx=13.28831, wy=0, wz=0, B=1.93128087, a=0.01750441,y=0, D=0, u = (12*16) / (12+16), state="ground")
    
    psi = wavefunction(CO, rydberg.equation, how(CO.re, CO.w, CO.u, 10))
    
    r, psiData = psi.graphData(startPoint=.8, endPoint=2)
    
    for n in range(psi.maxN):
        color = getColor()
        plot(r, psiData[n], "Wavefunction: " + str(n), lineColor = color)
        plot(r, [psi.energies[n] for value in r], "Energy Level: " + str(n), lineColor = color, dash="dash")
    
    print("Wavefunctions Built")
    
    psi.displayEnergy()
    
##########################################################################################    

#builds the Extended-Rydberg fit for the given radius and energy data
def buildPotential(R, E):
        
    print("Building Rydberg Potential Fit")
    rydberg.fitPotential(R, E)
    
    print("Graphing Rydberg Potential")
    ER = rydberg.graphData(0, R[-1] + 4)
    
    plot(ER[0], ER[1], "Extended Rydberg Fit")
        
##########################################################################################    

def loadFile():
    
    global fileLoad
        
    fileLoad = widgets.FileUpload(
        accept = ".txt",
        multiple = True,
        button_style = "info"
    )
        
    display(fileLoad)
    
    #reset graph data if new file is uploaded
    figure["data"] = []
        
    fileLoad.observe(parsePotentialFile, "value")

##########################################################################################    
    
def parsePotentialFile(file):
    
    clear_output()
    display(choice)
    loadFile()
    
    #Remove all unneded data so that only used data is accessible from the file dictionary
    file = file["new"][list(file["new"].keys())[0]]
    
    #check that file name is greater than five characters to ensure that [-4:] 
    #command will not cause an error when checking that file ends in ".txt"
    if(len(file["metadata"]["name"]) <= 5 or file["metadata"]["name"][-4:] != ".txt"):
            print("Warning! " + file["metadata"]["name"] + "' is not of the '.txt' type!")
    else:
        fileData = file["content"].decode("utf-8").split("\n")
                
        #set up empty lists to store bond distance and respective energy
        global R, E
        R = []
        E = []
        
        #use try catch in case that file is incorrectly formatted
        #and to catch the resulting error from parsing an incorrect file
        #assumed file is formatted with two columns of data separated by a space
        #1st column is bond distance, while 2nd column is the energy at that bond distance
        try:
            for lineNumber, line in enumerate(fileData):
                if(line.isspace() or line == ""):
                    continue

                line = line.split(" ")               

                R.append(float(line[0]))
                
                #Convert Energy from Hartrees to Wavenumbers to work with this computation
                E.append(float(line[-1]) * 2.1947463 * pow(10, 5))
                
            zeroingValue = -min(E)
            E = [e + zeroingValue for e in E]
        
        except ValueError:
            print("Warning!! Linenumber " + str(lineNumber) + " in the input file was incorrectly formatted!")
            print("Line " + str(lineNumber) + ": ''" + " ".join(line) + "''")
            print(line)
            return
        
        print("Potential Energy Surface Data Successfully Parsed!")
        plot(R, E, "Potential Energy Surface From File", mode="markers")
        
        buildPotential(R, E)
        buildWavefunction()
        iplot(figure)
          
##########################################################################################

#Handles logic for building potential curve from the diatomic constants
def diatomicConstantsLogic(buttonData):
    
    clear_output()
    display(choice)
    display(DCW)
    
    rkr.setDiatomicConstants(DCW.result)
    R, E = rkr.graphData(resolution = .5, endPoint=79)
    R.append(DCW.result.re)
    E.append(0)
    
    if not any(filter(np.isnan, R)):
        plot(R, E, "RKR Potential", mode="markers")
        buildPotential(R, E)
        buildWavefunction()
        iplot(figure)
    else:
        print("RKR Computation Failed!")
        print("Please Ensure Your Diatomic Constant Values Are Correct!")
        print("Or Contact zeri@chapman.edu To Report The Error")
    
##########################################################################################

def displayDiatomicConstantsMenu():
    
    global DCW
    DCW = diatomicConstantsWidget()
    
    display(DCW)
    display(startButton)
            
    startButton.on_click(diatomicConstantsLogic)
    
##########################################################################################

#Main Function used to exeute main decision logic of the program
def main(buttonData):

    #Refreshes screen to ensure buttons do not pile up on top of each other
    clear_output()
    display(choice)
    
    figure["data"] = []
    
    #check whether to display the load file menu or the get constatnts menu
    if(buttonData["new"] == "Use File"):
        loadFile()
    else:
        global startButton
        startButton = widgets.Button(description = "Start Calculation", button_style="Info")
        
        displayDiatomicConstantsMenu()

In [4]:
#Run Functions with Jupyter Widgets Overlay Here

print("Please choose how you like to set up the Extended-Rydberg Equation:")
choice = widgets.ToggleButtons(
    options = ["Use File", "Use Diatomic Constants"],
    button_style = "info",
    tooltips = ["", "Use Diatomic Constants"],
    value = None,
)

basisSize = widgets.IntText()

display(choice)

choice.observe(main, "value")

ToggleButtons(button_style='info', options=('Use File', 'Use Diatomic Constants'), tooltips=('', 'Use Diatomic…

FileUpload(value={}, accept='.txt', button_style='info', description='Upload', multiple=True)

Potential Energy Surface Data Successfully Parsed!
Building Rydberg Potential Fit
Preparing to Optimize the function
3.787349537078057
2.975618581172318
3.2580316000563365
Graphing Rydberg Potential


 90%|████████████████████████████████████████████████████████████████████████        | 9/10 [00:11<00:01,  1.85s/it]..\Comp_Chem_Package\wavefunction.py:115: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [00:14<00:00,  2.31s/it]



Generating Wavefunctions


100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.43it/s]


Wavefunctions Built


HTMLMath(value="<font size='5'>Energy Levels</font><br><font size='3'>Energy Levels $(n) \\qquad$ Energy in $c…